<a href="https://colab.research.google.com/github/cherry2202/Fake-News-Detection/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

# 检查是否有可用的GPU，如果有则使用GPU，没有则使用CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 创建一个Blip2Processor实例，从Hugging Face模型库中下载并加载预训练的Salesforce/blip2-opt-2.7b模型
# 处理器——预处理输入数据和后处理模型的输出
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

# 创建一个Blip2ForConditionalGeneration模型实例，它也将从Hugging Face的模型库中加载预训练的模型。
# device_map参数指定了模型应该在哪个设备上运行，torch_dtype=torch.float16指定了模型使用的精度。
# 模型——执行实际的学习和推理任务的核心
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT

# 下载图片的url
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 使用processor实例处理图像得到输入，pt代表返回PyTorch张量，to()将处理后的数据发送到指定设备并转换位torch.float16类型
inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

# 模型生成的文本的ID序列。在自然语言处理中，文本通常首先被转换为一系列ID，这些ID对应于词汇表中的单词或标记。
generated_ids = model.generate(**inputs)

# 使用处理器将生成的ID序列解码回文本。
# batch_decode方法将ID序列转换为可读的文本。skip_special_tokens=True参数告诉解码器跳过特殊标记，如开始、结束、填充等。
# batch_decode返回的是一个列表，即使只有一个生成的文本，所以这里使用索引[0]来获取第一个（也是唯一一个）文本。
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

two cats laying on a couch


In [ ]:
prompt = "Question: how many cats are there? Answer:"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

two
